# initializing and imports

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from transformers import pipeline
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import inspect
import pickle
import re
import time

# غیرفعال کردن هشدار MarkupResemblesLocatorWarning
import warnings
from bs4 import MarkupResemblesLocatorWarning
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)


In [3]:
hunting = 0  #  شیئی گروه برای ارسال پیام های یافت شده
previous_messages = []  #  زمان ارسال پیام های قبل برای جلوگیری از ارسال مجدد آن ها
sending_messages=[]  #  پیام های یافت شده را در این نگه میداریم تا زمان ارسال و بعد مجددا خالی اش می کنیم
chat_title=''
chat_id=''
timestamp = ''
driver = webdriver.Chrome()
driver.get("https://web.telegram.org/k/")



In [4]:
# بارگذاری مدل و توکنایزر ذخیره شده
model_path = "ai_model"
tokenizer_path = "ai_tokenizer"

# ایجاد pipeline برای دسته‌بندی توالی
classifier = pipeline("sentiment-analysis", model=model_path, tokenizer=tokenizer_path)


# Text Processing

### AI Model for distinguishing the text

In [52]:
# چک می کند که آیا پیام حاوی درخواستی برای معرفی وکیل یا مشاور مهاجرتی هست یا خیر
#  مدل قبل آموزش داده شده و همه جملات شامل مشاور یا وکیل را مورد بررسی قرار میده
# اگر جمله مرتبط تشخیص داده نشود برای چت خودم ارسالش می کند تا مطمئن باشم که درست عمل شده
def ai(text, min_score = 0.7):
    try:
        result = classifier(text)
    except:
        return False
    label = result[0]['label']
    score = result[0]['score']
    send_message_bot('LABEL: ' + label + ', SCORE: ' + str(score) + 'MESSAGE: ' + text, 'me')
    if(label == 'LABEL_1' and score > min_score):
        return True
    else:
        return False

### text_processing

In [7]:
#  تابع پردازش پیام ها
def text_processing(text):
    if(text == []):
        return False
    positive_expression =['منصوری','روادید',]

    negetive_expression =['تماس بگیرید','لطفا جهت کسب اطلاعات',
                          'خدمات ما شامل', 'لطفا جهت ثبت نام']
    
    necessary_expression = ['مشاور','وکیل','مشاوره',]
    
    for expression in negetive_expression:
        if expression in text:
            return False
        
    for expression in positive_expression:
        if expression in text:
            return True
    
    for expression in necessary_expression:
        if expression in text:
            if (ai(text)):
                return True
            else:
                return False
    
    return False
    

### remove_html_and_patterns

In [9]:
def remove_html_and_patterns(text):
    # مرحله 1: حذف تگ‌های HTML
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()

    # مرحله 2: حذف الگوهای خاص
    patterns = [
        r'\b\w*\.png\b',  # الگو برای *.png
        r'\bclass\w*\b',  # الگو برای class*
        r'\b\w*emoji\w*\b'  # الگو برای *emoji*
    ]
    
    for pattern in patterns:
        clean_text = re.sub(pattern, '', clean_text)

    # مرحله 3: حذف فاصله‌های اضافی
    clean_text = ' '.join(clean_text.split())

    return clean_text

### clean_message

In [10]:
#اینترها را قبل از ارسال پیام با اسپیس جایگزین کند
def clean_message(text):
    # Replace all newline characters with a space
    text = text.replace('\n', ' ')
    # Split the text by any whitespace and rejoin with a single space
    cleaned_text = ' '.join(text.split())
    return cleaned_text

# Dive in HTML

## Finding Things

### is_a_super_group_open

In [11]:
# بررسی می کند که آیا دکمه سوپر گروه باز است و درست یا غلط برمیگرداند
def is_a_super_group_open():
    try:
        if(driver.find_element(By.CSS_SELECTOR, "button.btn-icon.sidebar-close-button").find_element(By.CSS_SELECTOR,"span.tgico.button-icon")):
            return False
    except:
            return True

### find_list_of_all_chats

In [12]:
#  ساختن لیستی از کل چت های موجود    
def find_list_of_all_chats():
    close_super_group_button()
    return driver.find_elements(By.CSS_SELECTOR, "a.row.no-wrap.row-with-padding.row-clickable.hover-effect.rp.chatlist-chat.chatlist-chat-bigger.row-big") 


### find_list_of_all_sub_chats

In [13]:
#  ساختن لیستی از کل زیر چت های موجود    
def find_list_of_all_sub_chats():
    return driver.find_elements(By.CSS_SELECTOR,'ul.chatlist')[1] \
        .find_elements(By.CSS_SELECTOR,"a.row.no-wrap.row-clickable.hover-effect.rp.chatlist-chat.chatlist-chat-bigger.row-big")

### find_chat_title_and_id

In [14]:
#  یافتن عنوان و آیدی چت
def find_chat_title_and_id(chat):
    return [chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title")\
                .text ,\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-peer-id')]

### find_sub_chat_title_and_id

In [15]:
#  یافتن عنوان و آیدی زیرچت
def find_sub_chat_title_and_id(chat):
    return [chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title")\
                .text ,\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-peer-id')+\
            chat.find_element(By.CSS_SELECTOR,"div.row-title.no-wrap.user-title")\
                .find_element(By.CSS_SELECTOR,"span.peer-title") \
                .get_attribute('data-thread-id')]

### is_there_unread_message

In [16]:
#  چک می کند که آیا در این چت پیام خوانده نشده وجود دارد یا خیر
def is_there_unread_message(chat):
    try:
        if(chat.find_element(By.CSS_SELECTOR,"div.dialog-subtitle-badge.badge.badge-22.dialog-subtitle-badge-unread.is-visible.unread").text):
            return True  #  اگر این چت تگ پیام خوانده نشده داشته باشد مقدار تورو را برمگرداند وگرنه مقدار فالس را
    except:
        return False

### check_if_message_is_new

In [17]:
def check_if_message_is_new():
    global previous_messages, chat_id, timestamp
   
    is_new = True # آیا آی دی گروه جدید است؟
    for row in range(len(previous_messages)):
        if(previous_messages[row][0] == chat_id):
            is_new = False
            break
    if(is_new):
        previous_messages.append([])
        previous_messages[len(previous_messages) - 1].append(chat_id)
        row += 1

    for col in range(len(previous_messages[row])):
        if (previous_messages[row][col] == timestamp):
            return False
    
    previous_messages[row].append(timestamp)
    return True

## Do Something

### close_super_group_button

In [18]:
# این تابع دکمه بستن سوپر گروه ها را اگر باز شده باشند می زند تا بسته شود
def close_super_group_button():
        try:
                super_group_button = driver.find_element(By.CSS_SELECTOR, "button.btn-icon.sidebar-close-button")
                super_group_button.click()
        except:
                {}

### scroll_down

In [19]:
# با فراخوانی این تابع دکمه اسکرول داون در صورت وجود فشرده می شود
def scroll_down():
    scroll_down_button = driver.find_element(By.CSS_SELECTOR, ".btn-circle.btn-corner.z-depth-1.bubbles-corner-button.chat-secondary-button.bubbles-go-down.rp")
    try:
        scroll_down_button.click()
        time.sleep(3)
    except:
        {}

### clicking

In [20]:
def clicking(item):
    try:
        item.click()
        return False   # it means there is no problem in clicking the item
    except:
        time.sleep(10)
        try:
            item.click()
            return False   # it means there is no problem in clicking the item
        except:
            return True   # it means there is a problem in clicking the item

# Axiliary Functions

### warm_up

In [68]:
#  آرایه مسج هایی را که قبلا خوانده شده در صورت وجود بازیابی می کند و در صورت عدم وجود یک قالب اولیه از آن می سازد
def warm_up():
    global chat_id, chat_title
    global previous_messages
    
    try:
        # باز کردن فایل در حالت خواندن باینری
        with open('previous_messages.pkl', 'rb') as file:
            # بازیابی آرایه با استفاده از pickle
            previous_messages = pickle.load(file)
            
    except:
        chats = find_list_of_all_chats()
        for chat in chats:
            [chat_title, chat_id] = find_chat_title_and_id(chat)
            if not(redundent_chats(chat_id)):
                previous_messages.append([chat_id])
            

### save_previous_messages

In [64]:
#  اطلاعات پیام هایی که قبلا خوانده شده شده ذخیره می کنیم تا در صورت توقف برنامه
#  هنگام اجرای مجدد آن از ارسال پیامها تکراری جلوگیری شود
def save_previous_messages():
    global previous_messages
    
    # باز کردن فایل در حالت نوشتن باینری
    with open('previous_messages.pkl', 'wb') as file:
        # ذخیره آرایه با استفاده از pickle
        pickle.dump(previous_messages, file)


### redundent_chats

In [22]:
#  چت هایی که مطمئنیم نیازی به بررسی آن ها نیست
def redundent_chats(chat_id):
    if chat_id in ['5427116584','-4214311773', '6836232506',]:
        return True
    else:
        return False

### trim_rows

In [23]:
# only keep the last message info
def trim_rows(max_elements=50):
    global previous_messages
    
    for i in range(len(previous_messages)):
        if len(previous_messages[i]) > max_elements:
            # حذف عناصر از عنصر دوم تا تعداد مورد نظر
            previous_messages[i] = [previous_messages[i][0]] + previous_messages[i][len(previous_messages[i]) - max_elements + 1:]

# Comunication

### send_message_bot

In [24]:
# پیامی به چت خودم یا چت گروه هانتر می فرستند بر حسب اینکه پارامتر را هنگام فراخوانی چگونه ست کنیم
def send_message_bot(text, where = 'me'):
    token = '6836232506:AAEBdGfRFTbOVMePl2KZazyV4V0mglF29NM'
    chat_id = '7076137340'  #  آی دی خودم
    hunting_id = '-1002219036112'            #  آیدی گروه هانتینگ
    if(where != 'me'):
        chat_id = hunting_id
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    params = {'chat_id': chat_id, 'text': text}  # اصلاح 'hunting_id' به 'chat_id'
    response = requests.post(url, data=params)
    if response.status_code == 200:
        print(f"Message sent to {chat_id}: {text}")
    else:
        print(f"Failed to send message. Status code: {response.status_code}")


Message sent to -1002219036112: test


### send_message

In [25]:
def send_message(message):
    close_super_group_button()
    hunting.click()
    time.sleep(1)
    input_box = driver.find_element(By.CSS_SELECTOR, 'div.input-message-input')
    # پاک کردن هر مقدار موجود در اینپوت باکس
    input_box.clear()  # ممکن است این متد بر روی المان‌های contenteditable کار نکند، بسته به نوع مرورگر
    # ارسال مقدار مورد نظر به اینپوت باکس
    input_box.send_keys(message)
    time.sleep(2)
    
    send_button = driver.find_element(By.CSS_SELECTOR, 'div.btn-send-container')  #  انتخاب دکمه ارسال
    send_button.click()
    
    
    # # در صورت نیاز به ارسال اینتر در انتها
    # time.sleep(2)
    # input_box.send_keys(Keys.RETURN)

### log_error

In [26]:
def log_error(error_message):
    caller = inspect.stack()[1]
    try:
        with open('error_log.txt', 'a') as file:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            file.write(f"Time: {current_time}\n")
            file.write(f"Error in line <<< {caller.lineno} >>> of Function <<< {caller.function } >>>. \n")
            file.write(f"Error: {error_message}\n")
            file.write("\n" + "=*"*75 + "\n")
    except Exception as e:
        print(f"Failed to log error: {e}")


# Main Logic

### survey_the_chat

In [27]:
#  در اینجا ما برای یک چت پیام ها را بررسی می کنیم که آیا موردی که مد نظرمان باشد یافت می شود یا خیر
def survey_the_chat(chat):
    global sending_messages, hunting, chat_id, timestamp, chat_title
    
    caller_function = inspect.currentframe().f_back.f_code.co_name    
    
    [chat_title, chat_id] = find_chat_title_and_id(chat)
    # if chat_id == '-2219036112':   # گروه هانتینگ 
    #     hunting = chat
    #     return
    
    if(redundent_chats(chat_id)):
        # print('redundent chat <- <- <- ', chat_title)
        return

    if(not(is_there_unread_message(chat))):
        return;   
                
    # print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} necessary chat -> -> -> ", chat_title)         
    scroll_down()
    converstion_list = driver.find_elements(By.CSS_SELECTOR, "div.bubbles-group")   # پیام هر کاربر در تگ با این کلاس هست
    for conversation in converstion_list:
        try:
            user_name = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR, "span.peer-title").get_attribute("innerHTML"))
            message = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.translatable-message").get_attribute("innerHTML"))
            timestamp = conversation.find_element(By.CSS_SELECTOR,'.is-in').get_attribute('data-timestamp')
            # data_peer_id = conversation.find_element(By.CSS_SELECTOR,'.is-in').get_attribute('data-peer-id')
        except:
            continue
        if (check_if_message_is_new()):
            if(text_processing(message)):
                for_sending = f" **نام گروه :** <<< {chat_title} >>> **نام کاربر : ** <<< {user_name} >>> **پیام کاربر :** : {message}"
                sending_messages.append(for_sending)
                print(' ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ')
                print(' +++++++++++++++++++++++   WE GOT THE NEW FOLLOWING MESSAGE   +++++++++++++++++++++++++++ ')
                print(' ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ')
                print(for_sending)

### survey_the_sub_chats

In [28]:
def survey_the_sub_chats(mother_chat_title = 'unknown'):
    global sending_messages, chat_id, timestamp, chat_title
    cat_chats = find_list_of_all_sub_chats()
    for chat in cat_chats:
        if(True): #is_there_unread_message(chat)):              
            [sub_chat_title, chat_id] = find_sub_chat_title_and_id(chat)
            # print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ================subgroup {sub_chat_title} of mother group {mother_chat_title}==========")
            chat_title = f"{sub_chat_title} ({mother_chat_title})"
            if clicking(chat):
                continue
            time.sleep(2)  # کمی صبر کنید تا عملیات کلیک و هر اتفاق بعد از آن انجام شود
            scroll_down()
            time.sleep(1)
            sub_converstion_list = driver.find_elements(By.CSS_SELECTOR, "div.bubbles-group") # پیام هر کاربر در تگ با این کلاس هست
            for conversation in sub_converstion_list:
                try:
                    user_name = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.peer-title").text)
                    message = remove_html_and_patterns(conversation.find_element(By.CSS_SELECTOR,"span.translatable-message").text)
                    timestamp = conversation.find_element(By.CSS_SELECTOR,'div.is-in').get_attribute('data-timestamp')
                except Exception as e:
                    # print(f"**************************************Error: {e}")
                    continue
                    
                if (check_if_message_is_new()):
                    if(text_processing(message)):
                        for_sending = f" **نام گروه :** <<< {chat_title} >>> **نام کاربر : ** <<< {user_name} >>> **پیام کاربر :** : {message}"
                        sending_messages.append(for_sending)
    close_super_group_button()
                    

### once_check_all_groups

In [46]:
def once_check_all_groups():
    global hunting, sending_messages

    chats = find_list_of_all_chats()
    for chat in chats:
        [chat_title, chat_id] = find_chat_title_and_id(chat)
        
        if(clicking(chat)):
            continue
        
        time.sleep(2)
        if  (is_a_super_group_open()):
            survey_the_sub_chats(chat_title)        
        else:
            survey_the_chat(chat)          

    try:
        print(f"               ********************  {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  *************************")
        print('               *************************  END OF LOOP  ****************************')
        print('               ********************************************************************')
        print('')
        for message in sending_messages:
            message = clean_message(message)
            print('the current messages: +++++++++++++++++++++++++    ', message)
            send_message_bot(message, 'group')
            print('&&&&&&&&&&&&&&           THAT ONE SENT           &&&&&&&&&&&&&&&')
            print(f"Time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} :\n"+message)
            time.sleep(4)
        if len(sending_messages) == 0:
            print('^^^^^^^^^^^^^^^^^^^^^^   THERE WAS NO MESSAGE TO SENT   ^^^^^^^^^^^^^^^^^^^^^^^^^^')
        else:
            print('^^^^^^^^^^^^^^^^^^^^^^   ALL MESSAGES SUCCESSFULLY SENT   ^^^^^^^^^^^^^^^^^^^^^^^^^^')
            sending_messages = []  #  ریست کردن استورج جمع آوری پیام ها      
    except:
        print('%%%%%%%%%%%%%%%%%%%%%      there was a problem middle sending the messages and now we are in except part         %%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print(' the message with problem is:')
        print(message)
        {} 
    trim_rows()  #  حذف اضافات ماتریس اطلاعات مسج های قبلی
    refreshing()


### main

def main():
    # warm_up()
    while True:
        print('***********************  NEW ROUND  ****************************************')
        once_check_all_groups()


# RUNNING TIME

In [ ]:
def main():
    # warm_up()
    while True:
        print('               **************************  NEW ROUND  *****************************')

        once_check_all_groups()

if __name__ == "__main__":
    main()

#  تستی 

In [33]:
# sending_messages=[]
# previous_messages=[]


[]

In [61]:
save_previous_messages()

In [57]:
array_data

[['-1130337213',
  '1719866794',
  '1719866839',
  '1719867328',
  '1719867577',
  '1719867673',
  '1719868646',
  '1719868915',
  '1719869059',
  '1719869370',
  '1719871368',
  '1719871559',
  '1719871683',
  '1719871713',
  '1719871816',
  '1719871905',
  '1719872091',
  '1719872394',
  '1719872648',
  '1719872725',
  '1719872743',
  '1719872897',
  '1719870192',
  '1719873478',
  '1719873491',
  '1719873538',
  '1719873559',
  '1719884030',
  '1719884591',
  '1719884605',
  '1719884743',
  '1719885061',
  '1719885228',
  '1719885869',
  '1719886592',
  '1719886725',
  '1719886893',
  '1719886935',
  '1719888240',
  '1719888806',
  '1719888977',
  '1719889530',
  '1719889571',
  '1719889982',
  '1719890056',
  '1719890228',
  '1719890270',
  '1719890703',
  '1719890751',
  '1719890786'],
 ['-2219036112'],
 ['-1130337213'],
 ['-1358602703',
  '1719828667',
  '1719828940',
  '1719828994',
  '1719829218',
  '1719849715',
  '1719849896',
  '1719850654',
  '1719851022',
  '1719851171',
 